In [1]:
import copy
import math
import os
import sys
sys.path.append('../src/')
import warnings
warnings.filterwarnings('ignore')

from hydra.experimental import initialize, initialize_config_module, initialize_config_dir, compose
from IPython.display import display
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import torch
import torchvision
from torchvision.utils import make_grid
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from tqdm import tqdm

import custom
from kvt.builder import (
        build_callbacks,
        build_dataloaders,
        build_hooks,
        build_lightning_module,
        build_logger,
        build_loss,
        build_model,
        build_optimizer,
        build_scheduler,
        build_metrics,
)
import kvt.utils
from kvt.initialization import initialize as kvt_initialize

kvt_initialize()

In [5]:
with initialize(config_path='../config/'):
    config = compose(
        overrides=[
            '+trainer=exp001', 
#             '+augmentation=exp001', 
            '+dataset=default_dataset', 
        ]
    )
    # replace for notebook
#     config.dataset.dataset.params.data_dir = '../data/input'
#     config.trainer.evaluation.dirpath = '../data/output/predictions/oof'
#     config.trainer.evaluation.batch_size = batch_size

In [6]:
# build hooks
hooks = build_hooks(config)

# build model
model = build_model(config)

# build datasets
dataloaders = build_dataloaders(config)

Loaded object file: ../src/kvt/hooks/post_forward_hook.py
Loaded object file: ../src/kvt/hooks/model_builder_hook.py
Loaded object file: /opt/conda/lib/python3.7/site-packages/timm/models/resnet.py
---------------------------------------------------------------
dataset config: 
 {'name': 'RANZCRDataset', 'params': {'data_dir': '../data/input', 'csv_filename': 'preprocessed_train.csv', 'image_column': 'StudyInstanceUID', 'target_column': ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'], 'num_fold': 5, 'idx_fold': 0, 'mode': 'train', 'split': 'train'}}
Loaded object file: ../src/custom/transforms/ranzcr_transform.py
[ranzcr_transform] resize_to: (768, 768)
[ranzcr_transform] tta: 1
Loaded object file: ../src/custom/datasets/ranzcr.py
---------------------------------------------------------------
dataset config: 
 {'n